In [1]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, merge, Input, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import model_from_json
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
import random
import os
import cv2

from utils import Smooth, BatchGenerator

import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
df = pd.read_csv('driving_log.csv', skipinitialspace=True)
df.columns = ['center', 'left', 'right', 'dir', 'acc', 'nacc', 'speed']
df = df.drop((df[df['acc'] != 1]).index)
df = df.drop((df[df['speed'] < 30.1]).index)
print df['speed'].mean()
df = df.reset_index(drop=True)
df['dir'] = df['dir'] * 100
df.head()

30.1862628077


,center,left,right,dir,acc,nacc,speed
0,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-46.68390,1.0,0.0,30.14583
1,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-82.77869,1.0,0.0,30.10305
2,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-51.98100,1.0,0.0,30.18893
3,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-21.23459,1.0,0.0,30.19710
4,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,0.00000,1.0,0.0,30.18751


In [3]:
df = df.drop(['left', 'right', 'acc', 'nacc', 'speed'], axis=1)
df.columns = ['filename', 'dir']
df.head()

,filename,dir
0,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-46.68390
1,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-82.77869
2,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-51.98100
3,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-21.23459
4,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,0.00000


In [4]:
batch = 64
image_shape = (128, 128)

df_train, df_val = train_test_split(df, test_size=0.05, random_state=42)
train = BatchGenerator(df_train, batch, image_shape)
val = BatchGenerator(df_val, batch, image_shape)

print train.num, val.num

21417 1128


In [5]:
from keras.applications import VGG16

img_input = Input(shape=(image_shape[1], image_shape[0], 3))
base_model = VGG16(weights='imagenet', input_tensor=img_input, include_top=False)

In [6]:
x = base_model.output

x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1)(x)

model = Model(input=base_model.input, output=x)

model.compile(loss='mse', optimizer='adadelta', metrics=['accuracy'])

In [7]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 128, 128, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 128, 128, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 64, 64)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [8]:
zip(range(len(model.layers)), model.layers)

[(0, <keras.engine.topology.InputLayer at 0x109f0f7d0>),
 (1, <keras.layers.convolutional.Convolution2D at 0x154ba6bd0>),
 (2, <keras.layers.convolutional.Convolution2D at 0x154c5cd10>),
 (3, <keras.layers.pooling.MaxPooling2D at 0x1596e7f10>),
 (4, <keras.layers.convolutional.Convolution2D at 0x159713ed0>),
 (5, <keras.layers.convolutional.Convolution2D at 0x1597d7b10>),
 (6, <keras.layers.pooling.MaxPooling2D at 0x1597d79d0>),
 (7, <keras.layers.convolutional.Convolution2D at 0x159812ed0>),
 (8, <keras.layers.convolutional.Convolution2D at 0x159812050>),
 (9, <keras.layers.convolutional.Convolution2D at 0x159a28f50>),
 (10, <keras.layers.pooling.MaxPooling2D at 0x159a3afd0>),
 (11, <keras.layers.convolutional.Convolution2D at 0x159a759d0>),
 (12, <keras.layers.convolutional.Convolution2D at 0x159a75690>),
 (13, <keras.layers.convolutional.Convolution2D at 0x159ac0ad0>),
 (14, <keras.layers.pooling.MaxPooling2D at 0x159afa810>),
 (15, <keras.layers.convolutional.Convolution2D at 0x159

In [9]:
val_num = len(df_val.index)
X_valid = np.ndarray((val_num, image_shape[1], image_shape[0], 3), dtype=np.float32)
y_valid = np.ndarray((val_num, 1), dtype=np.float32)
for i, index in enumerate(df_val.index):
    img = cv2.imread(df_val.loc[index]['filename'])
    img = cv2.resize(img, image_shape)
    X_valid[i] = img
    y_valid[i] = df_val.loc[index]['dir']

def validation():
    loss, acc = model.evaluate(X_valid, y_valid, verbose=0)
    return acc, loss

validation()

(0.0017730496453900709, 747.60258451421214)

In [10]:
def train_one_batch(generator):
    acc = Smooth()
    loss = Smooth()
    batch_num = generator.num/batch
    pbar = tqdm(xrange(batch_num))
    for i in pbar:
        X, y = generator.next()
        _loss, _acc = model.train_on_batch(X, y)
        acc += _acc
        loss += _loss
        
        if i != batch_num-1:
            pbar.set_description("train loss: %.4f" % loss)
        else:
            pbar.set_description("train loss: %.4f\t valid loss: %.4f\t" % (loss, validation()[1]))

def transfer_learning(top_num=-4, epoch=10, optimizer='adadelta'):
    for layer in model.layers[:top_num]:
        layer.trainable = False

    for layer in model.layers[top_num:]:
        layer.trainable = True
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    
    for i in range(epoch):
        train_one_batch(train)

In [11]:
transfer_learning(18, epoch=6)

train loss: 105.4394	 valid loss: 172.1068	: 100%|██████████| 334/334 [01:25<00:00,  1.06s/it]


In [12]:
transfer_learning(14, epoch=4)

train loss: 128.3639	 valid loss: 150.6024	: 100%|██████████| 334/334 [01:38<00:00,  1.09s/it]


In [13]:
X_val, y_val = val.next()
y_pred = model.predict(X_val)
delta = y_val - y_pred

zip(y_val.tolist(), y_pred.tolist())

[([0.0], [1.7571333646774292]),
 ([2.286695957183838], [5.95246696472168]),
 ([-19.30561065673828], [1.0129343271255493]),
 ([-12.965689659118652], [-12.264935493469238]),
 ([0.0], [-2.5279927253723145]),
 ([0.0], [1.8169598579406738]),
 ([0.0], [-0.302773654460907]),
 ([0.0], [-2.2620153427124023]),
 ([0.0], [10.921895980834961]),
 ([50.90488815307617], [20.269895553588867]),
 ([0.0], [4.648195266723633]),
 ([-21.3906192779541], [-19.52005958557129]),
 ([45.78181838989258], [29.110355377197266]),
 ([-8.961886405944824], [-0.5977527499198914]),
 ([0.0], [-2.303544282913208]),
 ([0.0], [5.018388748168945]),
 ([24.748779296875], [2.4962007999420166]),
 ([-8.734809875488281], [-16.73863983154297]),
 ([-19.03080940246582], [-8.928788185119629]),
 ([0.0], [1.975947380065918]),
 ([0.0], [2.8204877376556396]),
 ([-26.86319923400879], [-4.699775218963623]),
 ([0.0], [1.337903618812561]),
 ([-63.47911071777344], [-19.936290740966797]),
 ([5.479771137237549], [22.628934860229492]),
 ([0.0], [13.

In [14]:
model.save('model.h5', overwrite=True)

In [15]:
from keras.applications import VGG16

img_input = Input(shape=(image_shape[1], image_shape[0], 3))
base_model = VGG16(weights='imagenet', input_tensor=img_input, include_top=False)

x = base_model.output

x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1)(x)

model = Model(input=base_model.input, output=x)

model.compile(loss='mse', optimizer='adadelta', metrics=['accuracy'])

with open('model.json', 'w') as f:
    f.write(model.to_json())